In [74]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller #used to auto-install chromedriver if current version of chrome is different
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pprint
from bs4 import BeautifulSoup
import re

chromedriver_autoinstaller.install()  
driver = webdriver.Chrome()
url = "https://twitter.com/explore"
driver.get(url)
# #click signin 
# time.sleep(2)
# sig_but = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div[1]/div[1]/div/div[3]/div[5]/a')
# time.sleep(1)
# sig_but.click()

# #sigin
# time.sleep(1)
# username = driver.find_element_by_xpath('//input[@name="text"]')
# time.sleep(1)
# username.send_keys('RAtestuser')
# time.sleep(1)
# username.send_keys(Keys.RETURN)

# #password 
# time.sleep(3)
# password = driver.find_element_by_xpath('//input[@name="password"]')
# time.sleep(1)
# password.send_keys('Reatest1.')
# time.sleep(1)
# password.send_keys(Keys.RETURN)

#write in search
time.sleep(1)
input_search = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[1]/div[1]/div/div/div/div/div[1]/div[2]/div/div/div/form/div[1]/div/div/label/div[2]/div/input')
input_search.send_keys("request for startup")
input_search.send_keys(Keys.ENTER)
time.sleep(2)

#latest 
driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[1]/div[2]/nav/div/div[2]/div/div[2]/a').click()


In [69]:
tweets = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')

In [70]:
def get_tweet_data(tweet):
    """Extract data from tweet card"""
    username = tweet.find_element_by_xpath('.//span').text
    try:
        handle = tweet.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = tweet.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = tweet.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = tweet.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    
    reply_cnt = tweet.find_element_by_xpath('.//div[@data-testid="reply"]').text
    if(reply_cnt == ''):
        reply_cnt = 0
    retweet_cnt = tweet.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    if(retweet_cnt == ''):
        retweet_cnt = 0
    like_cnt = tweet.find_element_by_xpath('.//div[@data-testid="like"]').text
    if(like_cnt == ''):
        like_cnt = 0
    
#     # get a string of all emojis contained in the tweet
#     """Emojis are stored as images... so I convert the filename, which is stored as unicode, into 
#     the emoji character."""
#     emoji_tags = card.find_elements_by_xpath('.//img[contains(@src, "emoji")]')
#     emoji_list = []
#     for tag in emoji_tags:
#         filename = tag.get_attribute('src')
#         try:
#             emoji = chr(int(re.search(r'svg\/([a-z0-9]+)\.svg', filename).group(1), base=16))
#         except AttributeError:
#             continue
#         if emoji:
#             emoji_list.append(emoji)
#     emojis = ' '.join(emoji_list)
    
    tweets = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweets    

In [71]:
for tweet in tweets:
    print(get_tweet_data(tweet))

('OilX', '@OilXs', '2022-01-26T12:39:31.000Z', 'OilX\n@OilXs\n·\n34mEarlier this week, we published our assessment of Nigeria´s highly anticipated Dangote Oil Refinery Company (DORC) startup for our clients. \n\nRequest a copy at research@oilx.co and learn more about OilX Research: https://hubs.li/Q012VYJq0\n\n#OOTT #downstream\n2\n4', 0, '2', '4')
("Paul O'Brien", '@seobrien', '2022-01-25T23:44:44.000Z', "Paul O'Brien\n@seobrien\n·\n13hProject for Hire Board? Upwork-like I suppose but more sophisticated and not based on skills. An RFP (request for proposal) board for startups.\n\nPost the project, details, and get offers to get it done.\n\n \n@heyalexfriedman\n  seems like something you might know. Maybe \n@FounderGigs_\n1", 0, 0, '1')
('MetaProp', '@MetaPropNYC', '2022-01-25T21:00:53.000Z', 'MetaProp\n@MetaPropNYC\n·\n16h***TOMORROW, January 26, 2022, 6:00 PM EST***\n\nThe MetaProp Accelerator at Columbia University is landing in #Miami for the first time ever with our 2021-2022 coho

In [75]:

# get all tweets on the page
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

i = 0
#while scrolling:
for i in range(5):
    page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    print("1")
    for card in page_cards[-15:]:
        print("2")
        tweet = get_tweet_data(card)
#         if tweet:
#             tweet_id = ' '.join(tweet)
#             if tweet_id not in tweet_ids:
#         tweet_ids.add(tweet_id)
        data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                time.sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

1
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


In [77]:
print(len(data))

51
